In [ ]:
!pip install fiftyone


In [ ]:
#second dataset download here

import fiftyone as fo
import fiftyone.zoo as foz

FOOD_CLASSES = [
    "Apple", "Banana", "Bell pepper", "Bread", "Cabbage", "Cake",
    "Carrot", "Cheese", "Chicken", "Ice cream", "Orange",
    "Pizza", "Potato", "Salad", "Shrimp", "Taco", "Tomato",
    "Watermelon", "French fries", "Hamburger", "Pasta",
    "Grape", "Peach", "Pear"
]


# Download the training split of Open Images V7, filtering for your classes
train_dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="train",
    label_types=["detections"],
    classes=FOOD_CLASSES,
    # Optional: limit the number of samples for a quicker start
    max_samples=10000,
    dataset_name="open-images-food-train",
)

# Download the validation split
val_dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="validation",
    label_types=["detections"],
    classes=FOOD_CLASSES,
    max_samples=2000,
    dataset_name="open-images-food-validation",
)

# You can launch the FiftyOne App to visualize and further curate your data

INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/open-images-v7/train' if necessary


INFO:fiftyone.utils.openimages:Downloading 10000 images


 100% |███████████████| 10000/10000 [19.9m elapsed, 0s remaining, 7.3 files/s]      


INFO:eta.core.utils: 100% |███████████████| 10000/10000 [19.9m elapsed, 0s remaining, 7.3 files/s]      


Dataset info written to '/root/fiftyone/open-images-v7/info.json'


...

INFO:fiftyone.zoo.datasets:Loading 'open-images-v7' split 'validation'


 100% |███████████████| 1781/1781 [11.2s elapsed, 0s remaining, 169.1 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 1781/1781 [11.2s elapsed, 0s remaining, 169.1 samples/s]      


Dataset 'open-images-food-validation' created


INFO:fiftyone.zoo.datasets:Dataset 'open-images-food-validation' created


In [ ]:
!pip install -q wandb torchviz fvcore grad-cam scikit-learn graphviz
!apt-get -q install graphviz



Reading package lists...
Building dependency tree...
Reading state information...
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
import fiftyone as fo

def convert_to_classification(dataset, food_classes, name):
    new_ds = fo.Dataset(name=name)

    for sample in dataset:
        if "ground_truth" not in sample:
            continue

        labels = [
            det.label
            for det in sample.ground_truth.detections
            if det.label in food_classes
        ]

        if not labels:
            continue

        sample["cls"] = fo.Classification(label=labels[0])
        new_ds.add_sample(sample)

    return new_ds


# Load the datasets
train_raw = fo.load_dataset("open-images-food-train")
val_raw   = fo.load_dataset("open-images-food-validation")

# Convert to classification
train_cls = convert_to_classification(
    train_raw, FOOD_CLASSES, "open-images-food-train-cls"
)

val_cls = convert_to_classification(
    val_raw, FOOD_CLASSES, "open-images-food-val-cls"
)

print("Train samples after filtering:", len(train_cls))
print("Val samples after filtering:", len(val_cls))


Train samples after filtering: 10000
Val samples after filtering: 1781


In [ ]:
train_cls.export(
    export_dir="/content/openimages_classification/train",
    dataset_type=fo.types.ImageClassificationDirectoryTree,
    label_field="cls"
)

val_cls.export(
    export_dir="/content/openimages_classification/val",
    dataset_type=fo.types.ImageClassificationDirectoryTree,
    label_field="cls"
)


 100% |█████████████| 10000/10000 [58.9s elapsed, 0s remaining, 219.6 samples/s]      


INFO:eta.core.utils: 100% |█████████████| 10000/10000 [58.9s elapsed, 0s remaining, 219.6 samples/s]      


 100% |███████████████| 1781/1781 [7.9s elapsed, 0s remaining, 194.2 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 1781/1781 [7.9s elapsed, 0s remaining, 194.2 samples/s]      


In [ ]:
from torchvision import datasets

train_full = datasets.ImageFolder("/content/openimages_classification/train")
val_full   = datasets.ImageFolder("/content/openimages_classification/val")

print("Train images:", len(train_full))
print("Val images:", len(val_full))
print("Classes:", len(train_full.classes))
print(train_full.classes)


Train images: 10000
Val images: 1781
Classes: 24
['Apple', 'Banana', 'Bell pepper', 'Bread', 'Cabbage', 'Cake', 'Carrot', 'Cheese', 'Chicken', 'French fries', 'Grape', 'Hamburger', 'Ice cream', 'Orange', 'Pasta', 'Peach', 'Pear', 'Pizza', 'Potato', 'Salad', 'Shrimp', 'Taco', 'Tomato', 'Watermelon']


In [ ]:
import os
os.environ["TORCH_COMPILE_DISABLE"] = "1"
os.environ["TORCHINDUCTOR_DISABLE"] = "1"
os.environ["TORCH_LOGS"] = ""


In [ ]:
import wandb
wandb.login()

wandb.init(
    project="vision-cw",
    name="resnet-openimages",
    config={
        "dataset": "openimages",
        "augmentation": True,
        "epochs": 30,
        "batch_size": 32,
        "lr": 1e-3
    }
)


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rashvin-rajanah-23 (rashvin-rajanah-23-ucl) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:

import os, random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from PIL import Image

from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

import wandb


device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

OUT_DIR = "/content/outputs"
os.makedirs(OUT_DIR, exist_ok=True)

wandb.init(
    project="vision-cw",
    name="resnet18_pretrained_openimages"
)


train_path = "/content/openimages_classification/train"
val_path   = "/content/openimages_classification/val"



train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    )
])


train_full = datasets.ImageFolder(train_path, transform=train_transform)
val_full   = datasets.ImageFolder(val_path, transform=val_transform)

num_classes = len(train_full.classes)
print("Classes:", num_classes)

train_dataset = train_full
val_dataset   = val_full


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Training images (pretrained): {len(train_dataset)}")
print(f"Validation images:            {len(val_dataset)}")

wandb.log({
    "train_images": len(train_dataset),
    "val_images": len(val_dataset),
    "num_classes": num_classes
})


model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

EPOCHS = 30
best_val_loss = float("inf")
patience, wait = 5, 0

train_losses, val_losses = [], []
train_accs, val_accs = [], []


for epoch in range(EPOCHS):
    model.train()
    correct, total, running_loss = 0, 0, 0

    for x, y in tqdm(train_loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        correct += (out.argmax(1) == y).sum().item()
        total += y.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = correct / total

    model.eval()
    v_loss, v_correct, v_total = 0, 0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            v_loss += criterion(out, y).item()
            v_correct += (out.argmax(1) == y).sum().item()
            v_total += y.size(0)

    val_loss = v_loss / len(val_loader)
    val_acc = v_correct / v_total

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "train_acc": train_acc,
        "val_acc": val_acc
    })

    print(f"Epoch {epoch+1}: TrainAcc={train_acc:.3f} ValAcc={val_acc:.3f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f"{OUT_DIR}/best_model_pretrained.pth")
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            break


plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.plot(train_losses); plt.plot(val_losses); plt.title("Loss")
plt.subplot(1,2,2); plt.plot(train_accs); plt.plot(val_accs); plt.title("Accuracy")
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/training_curves.jpg", dpi=300)
plt.close()

wandb.log({"training_curves": wandb.Image(f"{OUT_DIR}/training_curves.jpg")})


model.load_state_dict(torch.load(f"{OUT_DIR}/best_model_pretrained.pth", map_location=device))
model.eval()


y_true, y_pred = [], []
with torch.no_grad():
    for x, y in val_loader:
        preds = model(x.to(device)).argmax(1)
        y_true.extend(y.numpy())
        y_pred.extend(preds.cpu().numpy())

f1 = f1_score(y_true, y_pred, average="macro")
print("Macro F1:", f1)
wandb.log({"macro_f1": f1})

cm = confusion_matrix(y_true, y_pred, normalize="true")
ConfusionMatrixDisplay(cm).plot(cmap="Blues")
plt.savefig(f"{OUT_DIR}/confusion_matrix.jpg", dpi=300)
plt.close()

wandb.log({"confusion_matrix": wandb.Image(f"{OUT_DIR}/confusion_matrix.jpg")})


img_path = val_full.imgs[0][0]
img = Image.open(img_path).resize((224,224)).convert("RGB")
img_np = np.array(img) / 255.0
tensor = val_transform(img).unsqueeze(0).to(device)

cam = GradCAM(model=model, target_layers=[model.layer4[-1]])
heatmap = cam(input_tensor=tensor)[0]
vis = show_cam_on_image(img_np, heatmap, use_rgb=True)

plt.imshow(vis); plt.axis("off")
plt.savefig(f"{OUT_DIR}/gradcam.jpg", dpi=300)
plt.close()

wandb.log({"gradcam": wandb.Image(f"{OUT_DIR}/gradcam.jpg")})

wandb.finish()


Running on: cuda


Classes: 24
Training images (pretrained): 10000
Validation images:            1781


100%|██████████| 313/313 [02:35<00:00,  2.01it/s]


Epoch 1: TrainAcc=0.634 ValAcc=0.718


100%|██████████| 313/313 [02:35<00:00,  2.01it/s]


...

100%|██████████| 313/313 [02:34<00:00,  2.02it/s]


Epoch 8: TrainAcc=1.000 ValAcc=0.757


100%|██████████| 313/313 [02:35<00:00,  2.01it/s]


Epoch 9: TrainAcc=0.998 ValAcc=0.673
Macro F1: 0.7411628000956766


epoch,▁▂▃▄▅▅▆▇█
macro_f1,▁
num_classes,▁
train_acc,▁▅▇██████
train_images,▁
train_loss,█▄▂▁▁▁▁▁▁
val_acc,▄▆▆█▇▅█▇▁
val_images,▁
val_loss,▂▁▂▁▂▃▃▃█
epoch,9
macro_f1,0.74116


Running on: cuda
Number of classes: 24
✅ Loaded pretrained ResNet-18 weights from /content/outputs/best_model_pretrained.pth


ValueError: Sample larger than population or is negative

In [ ]:
from torchviz import make_dot
import torch
from torchvision import models

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
dummy = torch.randn(1, 3, 224, 224)

dot = make_dot(model(dummy), params=dict(model.named_parameters()))
dot.format = "png"
dot.render("/content/outputs/resnet18_architecture")

print("Saved to /content/outputs/resnet18_architecture.png")


Saved to /content/outputs/resnet18_architecture.png
